# Graph Attention Network using DGL - tutorial

## GAT architecture

In [1]:
# ! pip install dgl-cu101

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [17]:
class GATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        # equation (1)
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        # equation (2)
        self.attn_fc = nn.Linear(2 * out_dim, 1, bias=False)

    def edge_attention(self, edges):
        # edge UDF for equation (2)
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)
        return {'e': F.leaky_relu(a)}

    def message_func(self, edges):
        # message UDF for equation (3) & (4)
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # reduce UDF for equation (3) & (4)
        # equation (3)
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        # equation (4)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h}

    def forward(self, h):
        # equation (1)
        z = self.fc(h)
        self.g.ndata['z'] = z
        # equation (2)
        self.g.apply_edges(self.edge_attention)
        # equation (3) & (4)
        self.g.update_all(self.message_func, self.reduce_func)
        return self.g.ndata.pop('h')

In [18]:
class MultiHeadGATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim, num_heads, merge='cat'):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(GATLayer(g, in_dim, out_dim))
        self.merge = merge

    def forward(self, h):
        head_outs = [attn_head(h) for attn_head in self.heads]
        if self.merge == 'cat':
            # concat on the output feature dimension (dim=1)
            return torch.cat(head_outs, dim=1)
        else:
            # merge using average
            return torch.mean(torch.stack(head_outs))

In [19]:
class GAT(nn.Module):
    def __init__(self, g, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1 = MultiHeadGATLayer(g, in_dim, hidden_dim, num_heads)
        # Be aware that the input dimension is hidden_dim*num_heads since
        # multiple head outputs are concatenated together. Also, only
        # one attention head in the output layer.
        self.layer2 = MultiHeadGATLayer(g, hidden_dim * num_heads, out_dim, 1)

    def forward(self, h):
        h = self.layer1(h)
        h = F.elu(h)
        h = self.layer2(h)
        return h

## data

In [23]:
import time
import numpy as np
import networkx as nx

from dgl import DGLGraph
from dgl.data import citation_graph as citegrh

In [21]:
def load_cora_data():
    data = citegrh.load_cora()
    features = torch.FloatTensor(data.features)
    labels = torch.LongTensor(data.labels)
    mask = torch.BoolTensor(data.train_mask)
    g = data.graph
    # add self loop
    g.remove_edges_from(nx.selfloop_edges(g))
    g = DGLGraph(g)
    g.add_edges(g.nodes(), g.nodes())
    return g, features, labels, mask

In [22]:
g, features, labels, mask = load_cora_data()

In [26]:
# create the model, 2 heads, each head has hidden size 8
net = GAT(g,
          in_dim=features.size()[1],
          hidden_dim=8,
          out_dim=7,
          num_heads=2)

In [28]:
# create optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

# main loop
dur = []
for epoch in range(30):
    if epoch >= 3:
        t0 = time.time()

    logits = net(features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[mask], labels[mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
        epoch, loss.item(), np.mean(dur)))

/home/excelsiorcjh/miniconda3/envs/pytorch/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/excelsiorcjh/miniconda3/envs/pytorch/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 1.9461 | Time(s) nan
Epoch 00001 | Loss 1.9453 | Time(s) nan
Epoch 00002 | Loss 1.9446 | Time(s) nan
Epoch 00003 | Loss 1.9438 | Time(s) 0.1741
Epoch 00004 | Loss 1.9431 | Time(s) 0.1881
Epoch 00005 | Loss 1.9423 | Time(s) 0.1796
Epoch 00006 | Loss 1.9415 | Time(s) 0.1913
Epoch 00007 | Loss 1.9407 | Time(s) 0.1943
Epoch 00008 | Loss 1.9399 | Time(s) 0.1882
Epoch 00009 | Loss 1.9391 | Time(s) 0.1922
Epoch 00010 | Loss 1.9382 | Time(s) 0.1934
Epoch 00011 | Loss 1.9374 | Time(s) 0.1954
Epoch 00012 | Loss 1.9365 | Time(s) 0.1945
Epoch 00013 | Loss 1.9356 | Time(s) 0.1937
Epoch 00014 | Loss 1.9347 | Time(s) 0.1955
Epoch 00015 | Loss 1.9338 | Time(s) 0.1977
Epoch 00016 | Loss 1.9329 | Time(s) 0.1980
Epoch 00017 | Loss 1.9319 | Time(s) 0.1963
Epoch 00018 | Loss 1.9309 | Time(s) 0.1942
Epoch 00019 | Loss 1.9300 | Time(s) 0.1920
Epoch 00020 | Loss 1.9290 | Time(s) 0.1930
Epoch 00021 | Loss 1.9279 | Time(s) 0.1947
Epoch 00022 | Loss 1.9269 | Time(s) 0.1961
Epoch 00023 | Loss 1